<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/AllCF_Add_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import  drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
nltk.download('all')
from nltk.corpus import sentiwordnet as swn
device_name = tf.test.gpu_device_name()
print(device_name)
df = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/crowdFlower/cF_clean_our_combined.csv')
df = df.sample(frac=1,random_state=32)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

In [3]:
df.head()

,emotion,tweets
35975,1,Praying for love in a lap dance and paying in...
17590,2,trying to fix my internet connection gues...
38071,0,starting an account here on twitter
23304,2,hmmm wrong link ignore my tweet
26990,0,thanks before the major chop


In [27]:
df.emotion.unique()

array([1, 2, 0, 3])

In [28]:
no_labels=4
category_dict = {}
for i in range(no_labels):
  category_dict[i]=[0 for j in range(no_labels)]
  category_dict[i][i]=1
data_tweet = [str(x).lower().split() for x in df['tweets']]
data_cat = np.array([np.array(category_dict[x]) for x in df['emotion']])
print(data_cat[:5])

[[0 1 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 1 0]
 [1 0 0 0]]


In [29]:
print(data_tweet[:5])
print(data_cat[:5])

[['praying', 'for', 'love', 'in', 'a', 'lap', 'dance', 'and', 'paying', 'in', 'naivety', 'less', 'than', 'one', 'of', 'my', 'fav', 'songs'], ['trying', 'to', 'fix', 'my', 'internet', 'connection', 'guess', 'my', 'prayers', 'have', 'been', 'answered', 'and', 'i', 'wont', 'have', 'any', 'study', 'distractions', 'ugh'], ['starting', 'an', 'account', 'here', 'on', 'twitter'], ['hmmm', 'wrong', 'link', 'ignore', 'my', 'tweet'], ['thanks', 'before', 'the', 'major', 'chop']]
[[0 1 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 1 0]
 [1 0 0 0]]


In [30]:
# Parameters
Min_count = 0
Embedding_size = 200
Window_size = 5
Negative_sampling = 00

In [31]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)
w2v_sg.wv.init_sims(True)
w2v_cbow.wv.init_sims(True)

In [32]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.200d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [33]:
batch_size = 500
Max_input_size = max([len(x) for x in data_tweet])
data_tweet = [nltk.pos_tag(x) for x in data_tweet]
def getsent(word,tag):
  res=0
  try:
    x = swn.senti_synset(word+'.'+tag[0].lower()+'.01')
    res =  (x.pos_score()-x.neg_score())
  finally:
    return res 
data_tweet = [[(i[0],i[1],getsent(i[0],i[1])) for i in x] for x in data_tweet]

In [40]:
def vec_gen(w2v,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor=2):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(w2v.wv.word_vec(i[0]),i[1],i[2]) for i in x if i[0] in vocab]
        l=len(tweet)
        for i in range(l):
          if tweet[i][1][:2]=='JJ':
            for z in range(max(i-1,0),min(i+2,l)):
              if tweet[z][1][:2]=='NN':
                tweet[z]=((tweet[z][0]+tweet[i][0])/2,tweet[z][1])
          if tweet[i][1][:2]=='RB':
            for z in range(max(i-1,0),min(i+2,l)):
              if tweet[z][1][:2]=='VB':
                tweet[z]=((tweet[z][0]+tweet[i][0])/2,tweet[z][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = np.array(data_cat[k*(batch_size):(k+1)*(batch_size)])
      yield (temp,tempres)          
      
def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor=2):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(vocab[i[0]],i[1],i[2]) for i in x if i[0] in vocab.keys()]
        l=len(tweet)
        for i in range(l):
          if tweet[i][1][:2]=='JJ':
            for z in range(max(i-1,0),min(i+2,l)):
              if tweet[z][1][:2]=='NN':
                tweet[z]=((tweet[z][0]+tweet[i][0])/2,tweet[z][1])
          if tweet[i][1][:2]=='RB':
            for z in range(max(i-1,0),min(i+2,l)):
              if tweet[z][1][:2]=='VB':
                tweet[z]=((tweet[z][0]+tweet[i][0])/2,tweet[z][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = np.array(data_cat[k*(batch_size):(k+1)*(batch_size)])
      yield (temp,tempres)

In [41]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
sss.get_n_splits(data_tweet, data_cat)
X_train=[]
X_test=[]
y_train=[]
y_test=[]
counter=0
for train_index, test_index in sss.split(data_tweet, data_cat):
  print("TRAIN:", train_index, "TEST:", test_index)
  for i in train_index:
    X_train+=[data_tweet[i]]
    y_train+=[data_cat[i]]

  for j in test_index:
    X_test+=[data_tweet[j]]
    y_test+=[data_cat[j]]

 # X_train, X_test = data_tweet[train_index], data_tweet[test_index]
 # y_train, y_test = data_cat[train_index], data_cat[test_index]
print(X_train[:5])
print(y_train[:5])
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
sss.get_n_splits(X_train, y_train)
x_train=[]
X_val=[]
Y_train=[]
y_val=[]
for train_index1, test_index1 in sss.split(X_train, y_train):
  print("TRAIN:", train_index1, "TEST:", test_index1)
  for i in train_index1:
    x_train+=[X_train[i]]
    Y_train+=[y_train[i]]

  for j in test_index1:
    X_val+=[X_train[j]]
    y_val+=[y_train[j]]
X_train=x_train
y_train=Y_train
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))
print(len(X_val))
print(len(y_val))
print(X_train[:5])
print(y_train[:5])
result_table = [0,0,0]

TRAIN: [10470  6321 13813 ... 24250 39308  6334] TEST: [22946 37404 11668 ...  6796  7895 14585]
[[('the', 'DT', 0), ('goonies', 'NNS', 0), ('projected', 'VBN', 0), ('on', 'IN', 0), ('a', 'DT', 0), ('garage', 'NN', 0.0), ('door', 'NN', 0.0), ('with', 'IN', 0), ('my', 'PRP$', 0), ('friends', 'NNS', 0), ('amazing', 'VBG', 0)], [('17', 'CD', 0), ('again', 'RB', 0.0), ('tonight', 'NN', 0.0)], [('we', 'PRP', 0), ('finally', 'RB', 0.0), ('hit', 'VBD', 0.0), ('1000', 'CD', 0), ('friends', 'NNS', 0), ('thanks', 'NNS', 0.125)], [('lolzz', 'JJ', 0), ('public', 'JJ', 0), ('script', 'NN', 0.0), ('wait', 'NN', 0.0), ('n', 'NN', 0.0), ('watch', 'NN', 0.0)], [('night', 'NN', 0.0)]]
[array([0, 1, 0, 0]), array([1, 0, 0, 0]), array([0, 1, 0, 0]), array([0, 1, 0, 0]), array([1, 0, 0, 0])]
TRAIN: [28318 14444 15946 ... 11043 21820 15291] TEST: [23742   966  6886 ... 25081 24593 26654]
25600
25600
8000
8000
6400
6400
[[('geez', 'JJ', 0), ('15', 'CD', 0), ('years', 'NNS', 0.0), ('you', 'PRP', 0), ('d', 'VB

In [42]:
epochs = 30

In [43]:
from keras import backend as K
def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recalls = true_positives / (possible_positives + K.epsilon())
        return recalls
def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precisions = true_positives / (predicted_positives + K.epsilon())
        return precisions
def f1(y_true, y_pred):
    precisions = precision(y_true, y_pred)
    recalls = recall(y_true, y_pred)
    return 2*((precisions*recalls)/(precisions+recalls+K.epsilon()))

In [46]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dropout(0.2))
model_sg_bi.add(tf.keras.layers.Dense(64,activation='relu'))
model_sg_bi.add(tf.keras.layers.Dropout(0.2))
model_sg_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
sg_train_gen=vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size)
sg_val_gen=vec_gen(w2v_sg,vocab_sg,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_sg_bi = model_sg_bi.fit_generator(sg_train_gen,validation_data=sg_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_sg_bi.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)
result_table[1]=history[1]
print("SG_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
52/52 [==============================] - 72s 1s/step - loss: 1.2878 - accuracy: 0.3708 - f1: 0.0169 - precision: 0.1388 - recall: 0.0105 - val_loss: 1.1782 - val_accuracy: 0.4254 - val_f1: 0.0256 - val_precision: 0.6631 - val_recall: 0.0131
Epoch 2/30
52/52 [==============================] - 68s 1s/step - loss: 1.1715 - accuracy: 0.4287 - f1: 0.1266 - precision: 0.5984 - recall: 0.0732 - val_loss: 1.1434 - val_accuracy: 0.4602 - val_f1: 0.2439 - val_precision: 0.6023 - val_recall: 0.1531
Epoch 3/30
52/52 [==============================] - 68s 1s/step - loss: 1.1363 - accuracy: 0.4693 - f1: 0.2445 - precision: 0.6025 - recall: 0.1551 - val_loss: 1.1319 - val_accuracy: 0.4748 - val_f1: 0.3168 - val_precision: 0.5852 - val_recall: 0.2174
Epoch 4/30
52/52 [==============================] - 67s 1s/step - loss: 1.1156 - accuracy: 0.4829 - f1: 0.2999 - precision: 0.6116 - recall: 0.1995 - val_loss: 1.1214 - val_accuracy: 0.4848 - val_f1: 0.3435 - val_precision: 0.5742 - val_recall:

In [48]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dropout(0.2))
model_cbow_bi.add(tf.keras.layers.Dense(64,activation='relu'))
model_cbow_bi.add(tf.keras.layers.Dropout(0.2))
model_cbow_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
cbow_train_gen=vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size)
cbow_val_gen=vec_gen(w2v_cbow,vocab_cbow,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_cbow_bi = model_cbow_bi.fit_generator(cbow_train_gen,validation_data=cbow_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_cbow_bi.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[2]=history[1]
print("CBOW_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
52/52 [==============================] - 71s 1s/step - loss: 1.2851 - accuracy: 0.3618 - f1: 0.0146 - precision: 0.1490 - recall: 0.0085 - val_loss: 1.1795 - val_accuracy: 0.4237 - val_f1: 0.0161 - val_precision: 0.9121 - val_recall: 0.0082
Epoch 2/30
52/52 [==============================] - 64s 1s/step - loss: 1.1729 - accuracy: 0.4257 - f1: 0.0959 - precision: 0.6214 - recall: 0.0547 - val_loss: 1.1485 - val_accuracy: 0.4506 - val_f1: 0.2002 - val_precision: 0.6110 - val_recall: 0.1198
Epoch 3/30
52/52 [==============================] - 64s 1s/step - loss: 1.1382 - accuracy: 0.4691 - f1: 0.2327 - precision: 0.6076 - recall: 0.1453 - val_loss: 1.1315 - val_accuracy: 0.4737 - val_f1: 0.3198 - val_precision: 0.5792 - val_recall: 0.2211
Epoch 4/30
52/52 [==============================] - 66s 1s/step - loss: 1.1170 - accuracy: 0.4825 - f1: 0.3029 - precision: 0.6117 - recall: 0.2022 - val_loss: 1.1199 - val_accuracy: 0.4903 - val_f1: 0.3462 - val_precision: 0.5838 - val_recall:

In [49]:
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dropout(0.2))
model_glove_bi.add(tf.keras.layers.Dense(64,activation='relu'))
model_glove_bi.add(tf.keras.layers.Dropout(0.2))
model_glove_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
glove_train_gen=glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size)
glove_val_gen=glove_gen(vocab_glove,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_glove_bi = model_glove_bi.fit_generator(glove_train_gen,validation_data=glove_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_glove_bi.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[0]=history[1]
print("GLOVE_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
52/52 [==============================] - 34s 595ms/step - loss: 1.2386 - accuracy: 0.3981 - f1: 0.0869 - precision: 0.3625 - recall: 0.0559 - val_loss: 1.0829 - val_accuracy: 0.5186 - val_f1: 0.3853 - val_precision: 0.6212 - val_recall: 0.2794
Epoch 2/30
52/52 [==============================] - 30s 575ms/step - loss: 1.0595 - accuracy: 0.5337 - f1: 0.4244 - precision: 0.6375 - recall: 0.3196 - val_loss: 1.0279 - val_accuracy: 0.5531 - val_f1: 0.4739 - val_precision: 0.6413 - val_recall: 0.3760
Epoch 3/30
52/52 [==============================] - 30s 575ms/step - loss: 1.0160 - accuracy: 0.5614 - f1: 0.4771 - precision: 0.6553 - recall: 0.3760 - val_loss: 1.0194 - val_accuracy: 0.5545 - val_f1: 0.4806 - val_precision: 0.6501 - val_recall: 0.3814
Epoch 4/30
52/52 [==============================] - 30s 574ms/step - loss: 0.9870 - accuracy: 0.5785 - f1: 0.5029 - precision: 0.6721 - recall: 0.4021 - val_loss: 1.0013 - val_accuracy: 0.5649 - val_f1: 0.4704 - val_precision: 0.6783 -

IndexError: ignored

In [50]:
result_table[0]=history[1]
print("GLOVE_BI")

GLOVE_BI


In [51]:
#print("\tLstm\t\t\tBiLstm\t\t\tRnn")
print("glove",result_table[:1])
print("sg",result_table[1:2])
print("cbow",result_table[2:3])
# print(history_sg_bi.history)
# plt.title("Skip-Gram + Bi-LSTM accuracy, loss vs epochs Graph")
# plt.plot(history_sg_bi.history['loss'],c='b',label='loss')
# plt.plot(history_sg_bi.history['accuracy'],c='r',label='accuracy')
# plt.xlabel('epochs')
# plt.legend()
# # plt.yticks()
# plt.show()
# plt.title("GloVe + Bi-LSTM accuracy, loss vs epochs Graph")
# plt.plot(history_glove_bi.history['loss'],c='b',label='loss')
# plt.plot(history_glove_bi.history['accuracy'],c='r',label='accuracy')
# plt.xlabel('epochs')
# plt.legend()
# # plt.yticks()
# plt.show()

glove [0.5411249995231628]
sg [0.5026249885559082]
cbow [0.5164999961853027]
